In [5]:
!java -version

java version "1.8.0_421"
Java(TM) SE Runtime Environment (build 1.8.0_421-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.421-b09, mixed mode)


In [4]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

NameError: name 'os' is not defined

In [ ]:
%pip install -q tabula-py
%pip install selenium requests 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import re
import tabula
import pandas as pd
import numpy as np

### **Importing**

The following lines of code download the files for the breakdown of generation charge from MERALCO's Rate Archives.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://company.meralco.com.ph/news-and-advisories/rates-archives")

    # Click "Show more" until all items are loaded
    while True:
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-bordered load-more' and @title='Go to next page']"))
            )
            show_more_button.click()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            break  # Break the loop if no more "Show more" button is found

    rows = driver.find_elements(By.XPATH, "//tr[.//span[contains(text(), 'Generation')]]")

    # Prepare the download directory
    os.makedirs('pdf downloads', exist_ok=True)

    for row in rows:
        try:
            # Find the date in the same row
            date_element = row.find_element(By.XPATH, ".//td[contains(@class, 'views-field-field-date-created')]")
            date_text = date_element.text.strip()

            # Convert date text to "mm-yy" format
            match = re.search(r'(\w+)\s+(\d{4})', date_text)
            if match:
                month_str, year_str = match.groups()
                month = {
                    'January': '01', 'February': '02', 'March': '03', 'April': '04',
                    'May': '05', 'June': '06', 'July': '07', 'August': '08',
                    'September': '09', 'October': '10', 'November': '11', 'December': '12'
                }[month_str]
                date_formatted = f"{month}-{year_str[-2:]}"

                # Break the loop if the date is December 2017
                if month_str == 'December' and year_str == '2017':
                    print("Reached December 2017. Stopping downloads.")
                    break
            else:
                print(f"Date format not recognized for text: {date_text}")
                continue  # Skip this row if date format is not recognized

            # Find the PDF link
            pdf_link = row.find_element(By.XPATH, ".//a[contains(@class, 'btn-bordered-orange') and contains(@href, '.pdf')]")
            url = pdf_link.get_attribute('href')

            # Download the PDF
            response = requests.get(url)
            if response.status_code == 200:
                pdf_name = f"gc_table_{date_formatted}.pdf"
                with open(f'pdf downloads/{pdf_name}', 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Failed to download PDF from {url}, status code: {response.status_code}")

        except Exception as e:
            print(f"Error processing row: {e}")

finally:
    driver.quit()

Reached December 2017. Stopping downloads.


In [ ]:
#Create a new dataframe for each pdf file.

downloads_dir = "pdf downloads"

dataframes = {}

pdf_files = [f for f in os.listdir(downloads_dir) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    date_part = pdf_file.split('_')[-1].replace('.pdf', '')
    pdf_path = os.path.join(downloads_dir, pdf_file)

    df = tabula.read_pdf(pdf_path, stream=True, pages=1)[0]

    dataframes[f"df_{date_part}"] = df

### **Processing**

In [ ]:
def process_rows(df):

    supplier_index = df[df.iloc[:, 0].str.contains("SUPPLIERS", na=False)].index[0]
    df_new = df.iloc[supplier_index + 1:]

    df_new = df_new.reset_index(drop=True)

    wesm_index = df_new[df_new.iloc[:, 0].str.contains("WESM", na=False, case=False, regex=False)].index
    if not wesm_index.empty:
        wesm_index = wesm_index[0]
        df_new = df_new.iloc[:wesm_index + 1]

    # Reset index for the cleaned DataFrame
    df_new = df_new.reset_index(drop=True)

    # Extract the row containing "TOTAL" separately
    total_row = df[df.iloc[:, 0].str.contains("TOTAL", na=False, case=True, regex=False)]
    if not total_row.empty:
        total_row = total_row.reset_index(drop=True)
    else:
        total_row = None

    # Append the TOTAL row if it exists
    if total_row is not None:
        df_new = pd.concat([df_new, total_row], ignore_index=True)

    df_new = df_new[df_new.iloc[:, 0].str.contains(r'\d|WESM|TOTAL', na=False, case=True)]

    # Reset index for the cleaned DataFrame
    df_new = df_new.reset_index(drop=True)

    # Identify columns based on their content
    col_with_comma = None
    col_with_dot = None

    # Find the first column with a comma
    for i in range(1, len(df_new.columns)):
        if df_new.iloc[:, i].dtype == object:
            if col_with_comma is None and df_new.iloc[:, i].str.contains(',', na=False).any():
                col_with_comma = i

    # Find the last column with a dot
    for i in range(len(df_new.columns) - 1, -1, -1):
        if df_new.iloc[:, i].dtype == object:
            if df_new.iloc[:, i].str.contains('\.', na=False).any():
                col_with_dot = i
                break

    columns_to_include = [0]  # Always include the first column

    if col_with_comma is not None:
        columns_to_include.append(col_with_comma)

    if col_with_dot is not None and col_with_dot not in columns_to_include:
        columns_to_include.append(col_with_dot)

    if len(columns_to_include) == 1:
        # If only one column to include, default to the first two columns
        columns_to_include = [0, 1]

    df_new = df_new.iloc[:, columns_to_include]
    df_new.columns = ['Power Suppliers','kWh','Average Generation Cost']

    # Remove numbers in Power Suppliers
    df_new['Power Suppliers'] = df_new['Power Suppliers'].str.replace(r'^\d+\.\s*', '', regex=True)

    # Get the 'Generation Charge' value
    charge_row = df[df.iloc[:, 0].str.contains('CHARGE', case=False, na=False)]

    if not charge_row.empty:
        # Get the value from the last column
        rate_value = charge_row.iloc[:, -1].values[0]

        # Check if the value is NaN
        if pd.isna(rate_value):
            # If NaN, get the value from the second last column
            rate_value = charge_row.iloc[:, -2].values[0]
    else:
        rate_value = None

    df_new['Generation Charge'] = rate_value

    return df_new

In [ ]:
def add_date(df,key):
    df = process_rows(dataframes[key])

    end_part = key.split("_")[1]

    month_map = {
        '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
        '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
        '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
    }
    month, year = end_part.split('-')
    formatted_date = f"{month_map[month]}-{year}"

    # Add the new column to the dataframe
    df['Date'] = formatted_date

    return(df)

In [ ]:
# Initialize an empty list to store the processed dataframes
processed_dfs = []

for key in dataframes:
    df = dataframes[key]
    processed_df = add_date(process_rows(df), key)
    processed_dfs.append(processed_df)

# Concatenate all the processed dataframes into one big dataframe
big_df = pd.concat(processed_dfs, ignore_index=True)

In [ ]:
big_df['kWh'] = big_df['kWh'].str.replace(r'\s', '', regex=True)   # Remove all spaces
big_df['kWh'] = big_df['kWh'].str.replace(r'^\d+\.?\d*%', '', regex=True)  # Remove percentage and any preceding characters

big_df['Power Suppliers'] = big_df['Power Suppliers'].str.replace(r'\s*\d+\.?\d*%\s*$', '', regex=True)

big_df = big_df[~big_df['Power Suppliers'].str.contains("Subtotal", na=False, case=False)]

big_df

,Power Suppliers,kWh,Average Generation Cost,Generation Charge,Date
0,Quezon Power Phils Ltd. Co. (QPPL),"73,940,954",17.6352,6.3889,Apr-24
1,First Gas Power Corporation (FGPC) – Santa Rita,"499,668,979",6.7554,6.3889,Apr-24
2,FGP Corp. (FGP) – San Lorenzo,"274,353,355",6.5367,6.3889,Apr-24
3,First NatGas Power Corp. (FNPC) - San Gabriel,-,-,6.3889,Apr-24
4,San Buenaventura Power Ltd. Co. (SBPL),"286,480,000",6.9822,6.3889,Apr-24
5,Solar Philippines Tarlac Corp. (SPTC),"11,478,701",3.3122,6.3889,Apr-24
6,AC Energy (baseload),"139,200,000",3.9240,6.3889,Apr-24
7,AC Energy (midmerit),"45,936,000",4.4442,6.3889,Apr-24
8,Sual Power Inc (SPI) (midmerit),"106,720,000",5.1630,6.3889,Apr-24
9,Energy Development Corporation (EDC) (midmerit),"28,385,000",5.5294,6.3889,Apr-24


### Creating Supplier Dataframe

In [ ]:
unique_suppliers = big_df['Power Suppliers'].unique()

# Create a mapping of power suppliers to unique IDs
supplier_id_map = {supplier: id+1 for id, supplier in enumerate(unique_suppliers)}

# Create a new DataFrame from the mapping
supplier_df = pd.DataFrame(list(supplier_id_map.items()), columns=['Power Suppliers', 'Power Supplier ID'])

supplier_df

,Power Suppliers,Power Supplier ID
0,Quezon Power Phils Ltd. Co. (QPPL),1
1,First Gas Power Corporation (FGPC) – Santa Rita,2
2,FGP Corp. (FGP) – San Lorenzo,3
3,First NatGas Power Corp. (FNPC) - San Gabriel,4
4,San Buenaventura Power Ltd. Co. (SBPL),5
5,Solar Philippines Tarlac Corp. (SPTC),6
6,AC Energy (baseload),7
7,AC Energy (midmerit),8
8,Sual Power Inc (SPI) (midmerit),9
9,Energy Development Corporation (EDC) (midmerit),10


In [ ]:
# Create a mapping from Power Suppliers to Supplier IDs
supplier_mapping = dict(zip(supplier_df['Power Suppliers'], supplier_df['Power Supplier ID']))

# Replace names with IDs in big_df
big_df['Power Supplier ID'] = big_df['Power Suppliers'].map(supplier_mapping)

# Ensure IDs are not converted to floats
#big_df['Power Supplier ID'] = big_df['Power Supplier ID'].astype(int)

# Drop the old Power Suppliers column
big_df = big_df.drop(columns=['Power Suppliers'])

big_df.head()

,kWh,Average Generation Cost,Generation Charge,Date,Power Supplier ID
0,"73,940,954",17.6352,6.3889,Apr-24,1
1,"499,668,979",6.7554,6.3889,Apr-24,2
2,"274,353,355",6.5367,6.3889,Apr-24,3
3,-,-,6.3889,Apr-24,4
4,"286,480,000",6.9822,6.3889,Apr-24,5


In [ ]:
%pip install openpyxl

with pd.ExcelWriter("Historical_MERALCO_GC_Breakdown.xlsx", engine='openpyxl') as writer:
    big_df.to_excel(writer, sheet_name='Historical GC', index=False)
    supplier_df.to_excel(writer, sheet_name='Supplier IDs', index=False)

Note: you may need to restart the kernel to use updated packages.
